# TP1 Alejandro balladares 101118

In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors

import warnings
warnings.filterwarnings('ignore')

from datetime import datetime

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [3]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Punto 4

4 13. Realice una consulta en los contenidos actuales que le permita identificar algún
artículo que este vandalizado utilizando los datos de la revisión (⭐⭐)


In [4]:
id_logs ='1xbwXF1PqfUlCdPUQwdH1WDY6GrogSA3d'
downloaded = drive.CreateFile({'id': id_logs})
downloaded.GetContentFile('logs.csv')

df_logs = pd.read_csv('logs.csv')


In [5]:
id_contents = '1SarxhT65lTj5vztK-QnvrO625qMjCokB'

downloaded = drive.CreateFile({'id': id_contents})
downloaded.GetContentFile('contents.csv')

df_contents = pd.read_csv('contents.csv')

In [6]:
df_logs['timestamp'] = pd.to_datetime(df_logs['timestamp'])
df_contents['revision_timestamp'] = pd.to_datetime(df_contents['revision_timestamp'])

In [7]:
df_contents_ips = df_contents.dropna(subset=['revisor_ip']) #me quedo con los que tengan ip y no tengan usuario
df_contents_ips = df_contents_ips.drop(columns=['revisor_username', 'revisor_id'])

In [8]:
df_vandalismo = df_contents_ips.sort_values(by=['revision_timestamp'], ascending=False)
limite = df_vandalismo.iloc[944]['revision_timestamp'] #944 = solo fechas de "primero de septiembre" las más recientes | 3000 = del 30 de agosto para arriba
df_vandalismo = df_vandalismo.loc[df_vandalismo.revision_timestamp > limite]
#df_vandalismo

In [9]:
df_vandalismo['no_tiene_comentario'] = df_vandalismo['revisor_comment'].isna()
df_sin_comentarios = df_vandalismo.loc[df_vandalismo.no_tiene_comentario == True]
#df_sin_comentarios

In [10]:
df_logs_bloqueados = df_logs.loc[((df_logs.action == 'block') | (df_logs.action == 'reblock'))]
df_logs_bloqueados = df_logs_bloqueados.loc[df_logs.title.str.contains('Usuario', na=False)]
df_logs_bloqueados = df_logs_bloqueados.drop(columns=['contributor_username','contributor_id','contributor_ip'])

df_logs_bloqueados['title'] = df_logs_bloqueados['title'].str[8:]

df_logs_bloqueados

,item_id,timestamp,comment,logtype,action,title
5,6,2004-12-23 08:28:32+00:00,vandalismo grosero,block,block,212.55.30.157
287,288,2004-12-27 17:54:50+00:00,vandalismo grosero,block,block,201.241.228.15
288,289,2004-12-27 17:57:12+00:00,vandalismo en [[PlayStation]],block,block,201.255.50.32
290,291,2004-12-27 17:59:56+00:00,blanqueo de varios artículos,block,block,84.122.0.158
313,314,2004-12-28 01:38:59+00:00,spam sistemático de la web http://www.islamenl...,block,block,81.192.12.93
...,...,...,...,...,...,...
12188865,15587319,2021-09-03 02:05:46+00:00,[[WP:E|Violación de las reglas de etiqueta]],block,block,186.141.200.121
12188886,15587340,2021-09-03 02:24:41+00:00,[[WP:VN|Cuenta creada para vandalizar]],block,block,Galaxi066
12188909,15587363,2021-09-03 02:52:42+00:00,[[WP:VN|Cuenta creada para vandalizar]],block,block,Xx.tkpro.xx
12188924,15587378,2021-09-03 03:00:14+00:00,[[WP:VN|Vandalismo de páginas]],block,block,181.167.104.152


In [11]:
df_respuesta_4 = pd.merge(df_sin_comentarios, df_logs_bloqueados, left_on='revisor_ip', right_on='title', how='inner')
df_respuesta_4 #ediciones hechas por ips, sin ningún comentario y por gente que tiene historial de bloqueos, parece un buen criterio para empezar a buscar vandalismo

,title_x,id,namespace,revision_id,parent_revision,revision_timestamp,revisor_ip,revisor_comment,no_tiene_comentario,item_id,timestamp,comment,logtype,action,title_y
0,Idioma sumerio,145256,0,138053396,134996638.0,2021-09-01 13:37:58+00:00,37.133.13.83,NaN,True,15586169,2021-09-02 16:29:46+00:00,[[WP:SPAM|Spam]],block,block,37.133.13.83
1,Categoría:Orden Hospitalaria de San Juan de Dios,4905456,14,138052117,138052114.0,2021-09-01 12:12:07+00:00,119.12.240.144,NaN,True,15583372,2021-09-01 13:14:08+00:00,[[WP:VN|Vandalismo de páginas]],block,block,119.12.240.144
2,Aeropuerto Internacional de Macao,247203,0,138051758,136423530.0,2021-09-01 11:44:14+00:00,119.12.240.144,NaN,True,15583372,2021-09-01 13:14:08+00:00,[[WP:VN|Vandalismo de páginas]],block,block,119.12.240.144
3,Oasis del Tulum,1320976,0,138051557,138051552.0,2021-09-01 11:26:51+00:00,181.29.6.153,NaN,True,15032651,2021-02-11 14:38:12+00:00,[[WP:NSW|Sabotaje]],block,block,181.29.6.153
4,Oasis del Tulum,1320976,0,138051557,138051552.0,2021-09-01 11:26:51+00:00,181.29.6.153,NaN,True,15142678,2021-03-18 19:40:51+00:00,[[WP:VN|Vandalismo de páginas]]: Reincidente,block,block,181.29.6.153
5,Cuyo,82928,0,138051474,136819760.0,2021-09-01 11:18:36+00:00,181.29.6.153,NaN,True,15032651,2021-02-11 14:38:12+00:00,[[WP:NSW|Sabotaje]],block,block,181.29.6.153
6,Cuyo,82928,0,138051474,136819760.0,2021-09-01 11:18:36+00:00,181.29.6.153,NaN,True,15142678,2021-03-18 19:40:51+00:00,[[WP:VN|Vandalismo de páginas]]: Reincidente,block,block,181.29.6.153
7,La nona (película),1504466,0,138044994,138044981.0,2021-09-01 02:22:11+00:00,181.29.6.153,NaN,True,15032651,2021-02-11 14:38:12+00:00,[[WP:NSW|Sabotaje]],block,block,181.29.6.153
8,La nona (película),1504466,0,138044994,138044981.0,2021-09-01 02:22:11+00:00,181.29.6.153,NaN,True,15142678,2021-03-18 19:40:51+00:00,[[WP:VN|Vandalismo de páginas]]: Reincidente,block,block,181.29.6.153
9,La nona,1504340,0,138044925,137839109.0,2021-09-01 02:16:30+00:00,181.29.6.153,NaN,True,15032651,2021-02-11 14:38:12+00:00,[[WP:NSW|Sabotaje]],block,block,181.29.6.153
